# GPT-2 Daily Dialog Training

**Refactored notebook using training_lib**

Text generation training with comprehensive MLflow tracking.

In [ ]:
# Import the training library
import sys
sys.path.append('./training_lib')

from training_lib import (
    create_model, get_conversational_data, create_conversational_dataloader,
    UniversalTrainer, get_experiment_logger, print_model_summary
)

In [ ]:
# GPT-2 Configuration
CONFIG = {
    'model_name': 'gpt2',
    'dataset': 'blended_skill_talk',
    'epochs': 10,
    'batch_size': 64,
    'learning_rate': 5e-5,
    'max_conversations': 1000,
    'max_length': 512,
    'use_mlflow': True
}

print("GPT-2 Training Configuration:")
for key, value in CONFIG.items():
    print(f"  {key}: {value}")

In [ ]:
# Initialize experiment logger
logger = get_experiment_logger("GPT2", "Conversational")
logger.experiment_start()

# Create GPT-2 model and tokenizer
model, tokenizer = create_model(CONFIG['model_name'], pretrained=True)
print_model_summary(model, CONFIG['model_name'])

# Load conversational data
logger.info("Loading conversational dataset...")
conversations = get_conversational_data(
    CONFIG['dataset'], 
    max_conversations=CONFIG['max_conversations']
)

# Create data loader
train_loader = create_conversational_dataloader(
    conversations,
    tokenizer,
    batch_size=CONFIG['batch_size'],
    max_length=CONFIG['max_length']
)

logger.data_loaded(len(conversations))
print(f"\nDataset: {len(conversations):,} conversations, {len(train_loader)} batches")
if conversations:
    print(f"Sample: {conversations[0][:100]}...")

In [ ]:
# Create trainer for text generation
trainer = UniversalTrainer(
    model=model,
    model_name="GPT2",
    dataset_name="Conversational",
    epochs=CONFIG['epochs'],
    batch_size=CONFIG['batch_size'],
    learning_rate=CONFIG['learning_rate'],
    optimizer_name="AdamW",
    use_mlflow=CONFIG['use_mlflow'],
    tokenizer=tokenizer,
    input_size='max_length_512',
    use_pretrained=True,
    train_size=len(conversations)
)

# Start training
logger.info("Starting GPT-2 conversational training...")
print("\n" + "="*60)
print(" STARTING GPT-2 TRAINING")
print("="*60)

summary = trainer.train(train_loader)

print("\n" + "="*60)
print(" GPT-2 TRAINING COMPLETED!")
print("="*60)

In [ ]:
# Display results
if summary:
    print("\nGPT-2 TRAINING SUMMARY:")
    print("="*40)
    print(f"Status: {summary.get('status', 'Unknown')}")
    print(f"Total Time: {summary.get('total_time_minutes', 0):.1f} minutes")
    print(f"Best Token Accuracy: {summary.get('best_accuracy', 0):.4f}")
    print(f"Best Loss: {summary.get('best_loss', float('inf')):.4f}")
    print(f"Epochs: {summary.get('epochs_completed', 0)}/{CONFIG['epochs']}")
    print("="*40)

logger.info("GPT-2 conversational training completed!")